In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf



In [2]:
file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes


In [3]:
# Define the current directory and output data path
dir_cur = pathlib.Path(os.getcwd()).parent
output_data = dir_cur.joinpath("ssp_run")

# Change this line only when you receive a new run
run_folder = "sisepuede_summary_results_run_sisepuede_run_2025-07-21T13;50;17.070059"
run = output_data.joinpath(run_folder)

tableau = dir_cur.joinpath("Tableau/data")

In [4]:
ssp_output =pd.read_csv(run.joinpath("WIDE_INPUTS_OUTPUTS.csv"))
ssp_output.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,peru,0,369318.579985,688160.487871,39882.929854,316572.871666,1366.720811,3728.490914,130301.410464,...,4.06520,3.045533,2.906550,15.8623,1.69064,7.8884,120.7446,11.91872,17.830138,92.81
1,0,peru,1,375945.705353,699043.631490,40897.960041,315392.882011,1367.612978,3803.831286,127737.746149,...,4.11630,2.990550,3.099625,14.4220,1.62074,7.5453,112.1206,11.76626,17.452367,92.81
2,0,peru,2,399672.576875,704387.021263,40551.673946,330783.872932,1388.588126,3906.728073,123084.772217,...,4.13385,3.011300,3.431075,14.4531,1.69168,7.3091,121.2463,12.02988,17.691529,92.81
3,0,peru,3,452799.979433,700666.575896,39440.418124,328203.549775,1385.927255,3950.775600,122895.583417,...,4.13340,2.991850,3.275325,13.9065,1.73058,8.1242,121.8343,12.10522,18.420638,92.81
4,0,peru,4,480494.319528,708939.730845,39649.136940,333324.076629,1393.562031,3891.037107,123995.684662,...,4.16745,3.206333,3.310900,12.4729,1.67081,7.7002,126.0845,12.41426,18.582676,92.81


In [5]:
df_primary =pd.read_csv(run.joinpath("ATTRIBUTE_PRIMARY.csv"))
df_primary.head()

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,69069,0,6002,0
2,70070,0,6003,0


In [6]:
df_strategy =pd.read_csv(run.joinpath("ATTRIBUTE_STRATEGY.csv"))
df_strategy.tail()

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
66,4006,IP:ALL,Sectoral Composite - IPPU,All (unique by transformer) IPPU transformations,TX:IPPU:DEC_CLINKER|TX:IPPU:DEC_DEMAND|TX:IPPU...,0
67,6000,PFLO:INC_HEALTHIER_DIETS,Singleton - Default Value - PFLO: Change diets,NaN,TX:PFLO:INC_HEALTHIER_DIETS,0
68,6001,PFLO:INC_IND_CCS,Singleton - Default Value - PFLO: Industrial c...,NaN,TX:PFLO:INC_IND_CCS,0
69,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...,0
70,6003,PFLO:NDC_2020,NDC (2020 Start),NDC with start date of 2020; reflects original...,TX:AGRC:DEC_CH4_RICE_NDC2020_EST|TX:AGRC:DEC_E...,0


In [7]:
# Merge the dataframes to include primary and strategy information in the ssp_output
ssp_output = ssp_output.merge(  df_primary, how='left', on='primary_id')
ssp_output = ssp_output.merge(  df_strategy, how='left', on='strategy_id')

In [8]:
ssp_output.value_counts('strategy')

strategy
All Actions          36
NDC (2020  Start)    36
Strategy TX:BASE     36
Name: count, dtype: int64

In [9]:
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()
subsector_emission_fields

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_entc',
 'emission_co2e_subsector_total_fgtv',
 'emission_co2e_subsector_total_inen',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [10]:
ssp_output['year'] = ssp_output['time_period'] + 2015

In [11]:
# Melt ssp_output to long format for variables in subsector_emission_fields
ssp_long = ssp_output.melt(
    id_vars=['primary_id', 'strategy', 'strategy_id', 'year'],
    value_vars=subsector_emission_fields,
    var_name='Subsector',
    value_name='Emission'
)
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,emission_co2e_subsector_total_agrc,5.883682
1,0,Strategy TX:BASE,0,2016,emission_co2e_subsector_total_agrc,6.004029
2,0,Strategy TX:BASE,0,2017,emission_co2e_subsector_total_agrc,6.216007
3,0,Strategy TX:BASE,0,2018,emission_co2e_subsector_total_agrc,6.245386
4,0,Strategy TX:BASE,0,2019,emission_co2e_subsector_total_agrc,6.392131


In [12]:
ssp_long['Subsector'] = ssp_long['Subsector'].str.replace('emission_co2e_subsector_total_', '', regex=False)
ssp_long['Subsector'] = ssp_long['Subsector'].str.capitalize()
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agrc,5.883682
1,0,Strategy TX:BASE,0,2016,Agrc,6.004029
2,0,Strategy TX:BASE,0,2017,Agrc,6.216007
3,0,Strategy TX:BASE,0,2018,Agrc,6.245386
4,0,Strategy TX:BASE,0,2019,Agrc,6.392131


In [13]:
subsector_dict = {
    "Agrc": "Agriculture",
    "Frst": "Forest",
    "Lndu": "Land Use",
    "Lsmm": "Livestock Manure Management",
    "Lvst": "Livestock",
    "Soil": "Soil Management",
    "Wail": "Liquid Waste",
    "Waso": "Solid Waste",
    "Trww": "Wastewater Treatment",
    "Ccsq": "Carbon Capture and Sequestration",
    "Enfu": "Energy Fuels",
    "Enst": "Energy Storage",
    "Entc": "Energy Technology",
    "Fgtv": "Fugitive Emissions",
    "Inen": "Industrial Energy",
    "Scoe": "Stationary Combustion and Other Energy",
    "Trns": "Transportation",
    "Trde": "Transportation Demand",
    "Ippu": "IPPU",
    "Econ": "Economy",
    "Gnrl": "General"
}

ssp_long['Subsector'] = ssp_long['Subsector'].replace(subsector_dict)
ssp_long.value_counts('Subsector')

Subsector
Agriculture                               108
Carbon Capture and Sequestration          108
Energy Technology                         108
Forest                                    108
Fugitive Emissions                        108
IPPU                                      108
Industrial Energy                         108
Land Use                                  108
Livestock                                 108
Livestock Manure Management               108
Soil Management                           108
Solid Waste                               108
Stationary Combustion and Other Energy    108
Transportation                            108
Wastewater Treatment                      108
Name: count, dtype: int64

In [14]:
ssp_long = ssp_long.rename(columns={"primary_id": "Primary Id",
                                    "strategy": "Strategy",
                                    "strategy_id": "Strategy Id",
                                    "year": "Year"})

In [15]:
ssp_long

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agriculture,5.883682
1,0,Strategy TX:BASE,0,2016,Agriculture,6.004029
2,0,Strategy TX:BASE,0,2017,Agriculture,6.216007
3,0,Strategy TX:BASE,0,2018,Agriculture,6.245386
4,0,Strategy TX:BASE,0,2019,Agriculture,6.392131
...,...,...,...,...,...,...
1615,70070,NDC (2020 Start),6003,2046,IPPU,16.286635
1616,70070,NDC (2020 Start),6003,2047,IPPU,15.740388
1617,70070,NDC (2020 Start),6003,2048,IPPU,15.191633
1618,70070,NDC (2020 Start),6003,2049,IPPU,14.640229


In [16]:
ssp_long.to_csv(tableau.joinpath("emissions_peru_250821.csv"), index=False)